In [ ]:
pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import json
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

# load data from JSON file
data=pd.read_csv("/content/20200325_counsel_chat.csv")

# extract text and intent from data
texts = []
intents = []
for intent in data['questionID']:
    for text in intent['questionText']:
        texts.append(text)
        intents.append(intent['intent'])

# tokenize text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
encoded_texts = tokenizer.texts_to_sequences(texts)

# save tokenizer
import pickle

with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# pad sequences to have equal length
max_len = max([len(x) for x in encoded_texts])
padded_texts = pad_sequences(encoded_texts, maxlen=max_len, padding='post')

# create label encoder object
le = LabelEncoder()

# fit and transform the intents to integer labels
encoded_intents = le.fit_transform(intents)

# get the number of unique labels
num_intents = len(le.classes_)

# apply one-hot encoding to the integer labels
encoded_intents = tf.one_hot(encoded_intents, depth=num_intents)

# define model architecture
input_layer = Input(shape=(max_len,))
embedding_layer = Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=128, input_length=max_len)(input_layer)
lstm_layer = LSTM(128)(embedding_layer)
output_layer = Dense(num_intents, activation='softmax')(lstm_layer)
model = Model(inputs=input_layer, outputs=output_layer)

# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# train model
model.fit(padded_texts, encoded_intents, epochs=50, batch_size=16)

# save model
model.save('chatbot_model.h5')

TypeError: ignored

In [ ]:
import json
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

# load data from JSON file
with open('intents_final.json') as file:
    data = json.load(file)

# extract text and intent from data
texts = []
intents = []
for intent in data['intents']:
    for text in intent['text']:
        texts.append(text)
        intents.append(intent['intent'])

# tokenize text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)

# load saved model
model = load_model('chatbot_final.h5')

# define maximum sequence length
max_len = model.input_shape[1]

# create label encoder object
le = LabelEncoder()
le.fit(intents)

# create inverse mapping of label encoder for intent prediction
intent_mapping = {i: label for i, label in enumerate(le.classes_)}

# start chatbot interaction
print('Welcome to the chatbot! Type "quit" to exit.')
while True:
    # get user input
    user_input = input('You: ').lower().strip()
    
    # check if user wants to quit
    if user_input == 'quit':
        break
    
    # encode user input text
    encoded_input = tokenizer.texts_to_sequences([user_input])
    padded_input = pad_sequences(encoded_input, maxlen=max_len, padding='post')
    
    # predict intent
    intent_prob = model.predict(padded_input)[0]
    intent_idx = np.argmax(intent_prob)
    intent_label = le.inverse_transform([intent_idx])[0]
    
    # retrieve response
    for intent in data['intents']:
        if intent['intent'] == intent_label:
            response = np.random.choice(intent['responses'])
            print('Chatbot:', response)
            break

Welcome to the chatbot! Type "quit" to exit.
You: Hello
1/1 [==============================] - 1s 547ms/step
Chatbot: Good to see you again!
You: Where is HOD?
1/1 [==============================] - 0s 19ms/step
Chatbot: All engineering departments have only one hod XYZ who available on (Place name)
You: Where is cafateria?
1/1 [==============================] - 0s 19ms/step
Chatbot: Good to see you again!
You: Where is Cafetaria?
1/1 [==============================] - 0s 18ms/step
Chatbot: Our university has canteen with variety of food available
You: how many departments are there in JUIT
1/1 [==============================] - 0s 18ms/step
Chatbot: <a target="_blank" href="ADD YOU GOOGLE MAP LINK HERE"> here</a>


KeyboardInterrupt: ignored

In [ ]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from sklearn.preprocessing import LabelEncoder
from fuzzywuzzy import process
import random

# Load the dataset
data = pd.read_csv('/content/orchid - 20200325_counsel_chat.csv.csv')

# Convert the questionTitle to numerical labels using LabelEncoder
le = LabelEncoder()
le.fit(data['questionTitle'])
data['label'] = le.transform(data['questionTitle'])

# Tokenize the input data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data['questionText'])
vocab_size = len(tokenizer.word_index) + 1

# Pad the input sequences
maxlen = max([len(x.split()) for x in data['questionText']])
x = tokenizer.texts_to_sequences(data['questionText'])
x = pad_sequences(x, padding='post', maxlen=maxlen)

# Define the neural network model
model = Sequential()
model.add(Embedding(vocab_size, 32))
model.add(LSTM(32, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the neural network
model.fit(x, data['label'], epochs=50)

# Define a function to generate responses from the neural network
def generate_response(input_text):
    # Find the question in the dataframe that is closest to the input text
    closest_question = process.extractOne(input_text, data['questionText'])[0]
    # Get the corresponding label for the closest question
    label = data.loc[data['questionText'] == closest_question]['label'].values[0]
    # Get a random answer belonging to the corresponding questionTitle
    possible_answers = data.loc[data['label'] == label]['answerText'].values
    response = random.choice(possible_answers)
    return response

# Start the chatbot
print("Welcome to the chatbot!")
print("Type 'exit' to end the chatbot.")
while True:
    user_input = input("You: ")
    if user_input == 'exit':
        print("Chatbot: Goodbye!")
        break
    else:
        response = generate_response(user_input)
        print("Chatbot: ", response)


/usr/local/lib/python3.9/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


Epoch 1/50
67/67 [==============================] - 58s 791ms/step - loss: -1254.9589 - accuracy: 0.0019
Epoch 2/50
67/67 [==============================] - 46s 693ms/step - loss: -3506.1013 - accuracy: 0.0019
Epoch 3/50
67/67 [==============================] - 47s 697ms/step - loss: -4635.7725 - accuracy: 0.0019
Epoch 4/50
67/67 [==============================] - 47s 695ms/step - loss: -5581.3706 - accuracy: 0.0019
Epoch 5/50
67/67 [==============================] - 53s 790ms/step - loss: -6470.9893 - accuracy: 0.0019
Epoch 6/50
67/67 [==============================] - 46s 681ms/step - loss: -7331.6919 - accuracy: 0.0019
Epoch 7/50
67/67 [==============================] - 47s 708ms/step - loss: -8175.8477 - accuracy: 0.0019
Epoch 8/50
67/67 [==============================] - 46s 682ms/step - loss: -9008.9629 - accuracy: 0.0019
Epoch 9/50
67/67 [==============================] - 46s 694ms/step - loss: -9831.8994 - accuracy: 0.0019
Epoch 10/50
67/67 [==============================] - 45

In [ ]:
pip install fuzzywuzzy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
